# Environmental Master Code
## This is my master code for the Raspberry Pi.
##### Please do not edit, copy, or use without permission from owner, thank you.

### Import coding libraries to use

In [1]:
import busio ##black sensor library
import board ##black sensor library
import adafruit_ccs811 ##black sensor library
import smbus ##blue sensor libarary
import time ##library for general use
##import gspread ##google sheets library

### Code for Google Sheets - currently invalid

In [1]:
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("PiLog").sheet1

In [2]:
#rowcode for google sheets
row = ["hi", "this", "is", "KT"]
index = 1
sheet.insert_row(row, index)

In [3]:
#rowcode for google sheets
sheet.append_row(row)

In [5]:
#rowcode for google sheets
for i in range(10):
    row=[i,2*i,i-7]
    sheet.append_row(row)

### Code for the black external sensor

In [2]:
from board import *
i2c = board.I2C()   # uses board.SCL and board.SDA

In [3]:
#black sensor talks with adafruit
ccs =  adafruit_ccs811.CCS811(i2c)

In [4]:
#print black sensor readings
while True:
    print("CO2: ", ccs.eco2, " TVOC:", ccs.tvoc)
    print(ccs.data_ready)
    time.sleep(3)

### Code for the blue external sensor

In [5]:
#read blue sensor data and calculate the external temperature
def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [5]:
#contact and sign into Adafruit with the .env
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME")
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

In [7]:
#bme 280
from bme280 import BME280
bme280 = BME280()

### Black and blue sensor master print

In [7]:
#read feeds, create metadata, create letter variables, print data
exterFeed = aio.feeds("external")
tempFeed = aio.feeds("temperature")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
co2Feed = aio.feeds("co2")
tvocFeed = aio.feeds("tvoc")

metadata = {"lat": 40.262726,
            "lon": -74.94198,
            "ele": 0,
            "created_at": None}
while True:
    e = readTemp()
    t = bme280.get_temperature()
    f = e * (9./5.) +32
    q = t * (9./5.) +32 
    h = bme280.get_humidity()
    p = bme280.get_pressure()
    co2 = ccs.eco2
    tvoc = ccs.tvoc
    aio.send_data(exterFeed.key, f, metadata)
    aio.send_data(tempFeed.key, q, metadata)
    aio.send_data(humidFeed.key, h, metadata)
    aio.send_data(pressFeed.key, p, metadata)
    aio.send_data(co2Feed.key, co2, metadata)
    aio.send_data(tvocFeed.key, tvoc, metadata)
    print(f, ":°Fahrenheit|")
    print(q, ":°Fahrenheit|")
    print(h, ":Humidity|")
    print(p, ":Barometric Pressure|")
    print(co2, ": PPM (CO2)|")
    print(tvoc, ":PPB (TVOC)|")
    print()
    time.sleep(15)